<a href="https://colab.research.google.com/github/helomelo1/ZtH_with_Karpathy/blob/main/makemore_using_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
words = open('names.txt', 'r').read().splitlines()
print(words[:5])

['emma', 'olivia', 'ava', 'isabella', 'sophia']


In [20]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

# print(itos)

In [21]:
block_size = 3
X, y = [], []

for w in words:
    print('\n', w)
    context = [0] * block_size

    for ch in w + '.':
        idx = stoi[ch]
        X.append(context)
        y.append(idx)

        # print(''.join(itos[i] for i in context), '->', itos[idx])

        context = context[1:] + [idx]

X = torch.tensor(X)
y = torch.tensor(y)

Streaming output truncated to the last 5000 lines.

 lakyn

 lamarius

 lanny

 lathaniel

 layn

 lazzaro

 ledgen

 leeandre

 legendary

 legennd

 leibish

 leighland

 lendell

 lennard

 leny

 leonhard

 lesandro

 levee

 liang

 life

 lindell

 liyan

 llewelyn

 lochlen

 locklan

 locklen

 lockwood

 locryn

 lonan

 louka

 lovensky

 ludovico

 lukaz

 lumi

 lumiere

 luqmaan

 luxe

 lynix

 maan

 macaulay

 machi

 maciah

 mackinnon

 maclyn

 macson

 macyn

 maddoxx

 mahil

 maier

 mainor

 majik

 makarios

 makeen

 makeo

 malex

 malhar

 malikiah

 malyk

 mandela

 mani

 manil

 manmeet

 mansur

 manuia

 marcellis

 mareon

 mari

 markai

 markeese

 markhi

 markian

 marrion

 marsden

 martyn

 masir

 maslah

 masood

 matas

 mati

 mattison

 matua

 maverix

 mavi

 mavin

 maximilio

 maxsen

 maxston

 maxyn

 mayel

 mayhem

 mayron

 maysin

 mcclane

 mcguire

 mekko

 melody

 merci

 merrit

 mesai

 mesias

 messyah

 mete

 miccah

 mic

In [22]:
print(X.shape)
print(y.shape)

torch.Size([228146, 3])
torch.Size([228146])


In [23]:
c = torch.randn((27, 2))
embedding = c[X]
print(embedding.shape)

torch.Size([228146, 3, 2])


In [24]:
w1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [25]:
h = torch.matmul(embedding.view(-1, 6), w1) + b1
h = torch.tanh(h)

In [26]:
w2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [27]:
logits = torch.matmul(h, w2) + b2
logits.shape

torch.Size([228146, 27])

In [28]:
counts = logits.exp()

In [29]:
probs = counts / counts.sum(1, keepdim=True)
probs.shape

torch.Size([228146, 27])

In [30]:
# REARRANGING STUFF BELOW

In [31]:
g = torch.Generator().manual_seed(0)
c = torch.randn((27, 2), generator=g)

w1 = torch.randn((6, 100))
b1 = torch.randn(100)

w2 = torch.randn((100, 27))
b2 = torch.randn(27)

parameters = [c, w1, b1, w2, b2]

In [32]:
sum(p.nelement() for p in parameters)

3481

In [33]:
for p in parameters:
    p.requires_grad = True

In [37]:
for i in range(1000):
    idx = torch.randint(0, X.shape[0], (32, ))

    emb = c[X[idx]]

    h = torch.matmul(emb.view(-1, 6), w1) + b1
    h = torch.tanh(h)

    logits = torch.matmul(h, w2) + b2
    loss = F.cross_entropy(logits, y[idx])

    for p in parameters:
        p.grad = None
    loss.backward()

    for p in parameters:
        p.data += -0.1 * p.grad

    if i % 100 == 0:
        print(f'Epoch {i}: {loss.item()}')

# print(loss.item())

Epoch 0: 2.6260173320770264
Epoch 100: 2.3917226791381836
Epoch 200: 2.592360496520996
Epoch 300: 2.740023136138916
Epoch 400: 2.4256045818328857
Epoch 500: 2.4590609073638916
Epoch 600: 2.3843541145324707
Epoch 700: 2.5431745052337646
Epoch 800: 2.5646419525146484
Epoch 900: 2.745731830596924
